In [1]:
def clip(tmp1):
    import subprocess
    import shlex  # 导入 shlex 模块
    # 使用 shlex.quote 来转义 inp 字符串
    tmp2 = str(tmp1)
    safe_str = shlex.quote(tmp2)
    subprocess.run('echo {} | wclip'.format(safe_str), shell=True)  

def cvin(k):
    clip(In[k])
    
import numpy as np
import matplotlib.pyplot as plt

import copy #复制方法
#格式化输出
np.set_printoptions(precision=3, suppress=True, linewidth=100)

#### 无模型的强化学习:
动态规划算法要求马尔可夫决策过程是已知的，但通常状态转移概率是无法写出来的，只能和环境进行交互，采样数据，被称为**无模型的强化学习（model-free reinforcement learning）**
无模型的强化学习中的两大经典算法：
- Sarsa
- Q-learning

它们都是基于**时序差分（temporal difference，TD）**的强化学习算法
引入一组概念：在线策略学习和离线策略学习:
- 在线策略学习:要求使用在当前策略下采样得到的样本进行学习，一旦策略被更新，当前的样本就被放弃了
- 离线策略学习:使用经验回放池将之前采样得到的样本收集起来再次利用，离线策略学习往往能够更好地利用历史数据，并具有更小的样本复杂度

## 时序差分算法：
时序差分结合了蒙特卡洛和动态规划算法
- 与蒙特卡洛相似：可以从样本数据中学习，不需要事先知道环境
- 与动态规划相似：根据贝尔曼方程的思想，利用后续状态的价值估计来更新当前状态的价值估计

主要使用公式从$V\left(s_{t}\right) \leftarrow V\left(s_{t}\right)+\alpha\left[G_{t}-V\left(s_{t}\right)\right]$变形而来，其中$\alpha$为$ \frac{1}{N(s)}$(取常数不再像蒙特卡洛严格取期望)，便得到了时序差分的公式：
$$
V\left(s_{t}\right) \leftarrow V\left(s_{t}\right)+\alpha\left[r_{t}+\gamma V\left(s_{t+1}\right)-V\left(s_{t}\right)\right]
$$
其中$r_{t}+\gamma V\left(s_{t+1}\right)-V\left(s_{t}\right)$被称作时序差分的**误差(error)**
- $G_t$ 是基于实际经验的完整回报，需要等待整个轨迹完成后才能计算。
- $R_t + \gamma V(S_{t+1})$ 是一个估计的回报，它结合了实际观察到的奖励和对未来回报的估计。这允许我们在每一步之后立即更新值函数，而不需要等待整个轨迹完成。

#### |附录：使用$r_{t}+\gamma V\left(s_{t+1}\right)$代替$G_T$原因：
$V(s) = \mathbb{E}\left[G_{t} \mid S_{t}=s\right]$

$= \mathbb{E}\left[R_{t}+\gamma R_{t+1}+\gamma^{2} R_{t+2}+\ldots \mid S_{t}=s\right]$

$= \mathbb{E}\left[R_{t}+\gamma\left(R_{t+1}+\gamma R_{t+2}+\ldots\right) \mid S_{t}=s\right]$

$= \mathbb{E}\left[R_{t}+\gamma G_{t+1} \mid S_{t}=s\right]$

$= \mathbb{E}\left[R_{t}+\gamma V\left(S_{t+1}\right) \mid S_{t}=s\right]$

因此蒙特卡洛方法将上式第一行作为更新的目标，而时序差分算法将上式最后一行作为更新的目标
1. **实际的回报 $G_t$**:
   - $G_t$ 是从时间 $t$ 开始到轨迹结束的累积奖励的总和。它是实际观察到的奖励序列的折现和。
   - 具体地说，$G_t = R_{t} + \gamma R_{t+1} + \gamma^2 R_{t+2} + \ldots$。这是一个确定的值，因为它是基于实际经验（即智能体与环境的交互）得到的。
   - 为了使用蒙特卡洛方法估计 $V(s)$，我们需要等待整个轨迹完成，然后计算 $G_t$。

2. **当前的奖励 $R_t$ 和下一个状态的估计值 $V(S_{t+1})$**:
   - 这是时序差分方法的核心。它结合了当前的奖励和下一个状态的值函数估计来更新当前状态的值函数。
   - 具体地说，更新目标是 $R_t + \gamma V(S_{t+1})$。这是一个估计的回报，因为它包括了对未来回报的估计（即 $V(S_{t+1})$）。
   - 与蒙特卡洛方法不同，时序差分方法不需要等待整个轨迹完成。它可以在每一步之后立即更新值函数。



### 1.Sarsa算法

$$
Q\left(s_{t}, a_{t}\right) \leftarrow Q\left(s_{t}, a_{t}\right)+\alpha\left[r_{t}+\gamma Q\left(s_{t+1}, a_{t+1}\right)-Q\left(s_{t}, a_{t}\right)\right]
$$